# Introduction to OpenFermion

In this tutorial we will learn how to use OpenFermion to represent and manipulate strongly-correlated Hamiltonians, and to interface it with Cirq.
To install OpenFermion, please see the instructions at https://github.com/quantumlib/OpenFermion
To install the OpenFermion-Cirq interface, please see the instructions at https://github.com/quantumlib/OpenFermion-Cirq.

(Some of the code in this tutorial is adapted from tutorial 4 of OpenFermion-Cirq.) 

1. What is OpenFermion?
1. Operators
    1. The QubitOperator class
    1. The FermionOperator class
    1. The Jordan-Wigner transformation
1. Quantum Chemistry
    1. The MolecularData class
    1. The InteractionOperator class
1. The OpenFermion-Cirq interface
1. Exercise: the unitary coupled cluster ansatz
    

# What is OpenFermion?

OpenFermion is a package that allows for the easy representation and manipulation of operators on strongly-correlated fermionic, bosonic, and qubit systems. This in turn allows it to act as an interface between computational chemistry software and quantum circuit design packages such as cirq. It also contains various methods for generating quantum circuits of use for quantum chemistry/strongly correlated materials.

# Operators

OpenFermion provides many convenient methods for representing the various operators that appear in strongly-correlated physics and chemistry problems. In particular, OpenFermion aims for these operators to be human-readable, and to not require storing an exponentially large matrix.

## The QubitOperator class

We have repeatedly encountered the Pauli operators on $N$ qubits, $\{I,X,Y,Z\}^{\otimes N}$. OpenFermion represents these, and linear combinations of Pauli operators, using the QubitOperator class. Internally, the data about the operator is stored as a dictionary, with the names of individual Pauli operators used as keys.

QubitOperators are initialized as single Pauli terms separated by space, and each Pauli term is written index-last. An optional coefficient may be provided to multiply the term:

In [1]:
from openfermion import QubitOperator
qop1 = QubitOperator("X1 Y2", 3.0)
print(qop1)

3.0 [X1 Y2]


QubitOperators may be added, multiplied, and multiplied by constants to produce other QubitOperators 

In [2]:
qop2 = QubitOperator("Z1 Z2")
print(qop1 + 3 * qop2)
print(qop1 * qop2)

3.0 [X1 Y2] +
3.0 [Z1 Z2]
(3+0j) [Y1 X2]


**Exercise 1:** Verify that QubitOperators on a single qubit obey the Pauli operator commutation relations.

In [3]:
qopx0 = QubitOperator('X0', 1.0)
qopy0 = QubitOperator('Y0', 1.0)
qopz0 = QubitOperator('Z0', 1.0)

Square to identity:

In [4]:
qopx0 * qopx0

1.0 []

Commutation:

In [5]:
qopy0 * qopz0 - qopz0 * qopy0

2j [X0]

Anti-commutation:

In [6]:
qopy0 * qopz0 + qopz0 * qopy0

0

Paulis in acting on different qubits:

In [7]:
qopx1 = QubitOperator('X1', 1.0)
qopy1 = QubitOperator('Y1', 1.0)
qopz1 = QubitOperator('Z1', 1.0)

Product of Paulis:

In [8]:
qopx0 * qopx1

1.0 [X0 X1]

Commutator:

In [9]:
qopz0 * qopy1 - qopy1 * qopz0

0

Anti-commutator

In [10]:
qopz0 * qopy1 + qopy1 * qopz0

2.0 [Z0 Y1]

OpenFermion additionally has methods to convert QubitOperators into sparse matrix form, or to calculate the eigenspectrum.

In [11]:
from openfermion import eigenspectrum, get_sparse_operator
print(eigenspectrum(qop1 + 3*qop2))
matrix = get_sparse_operator(qop1 + 3*qop2).todense()
print()
print(matrix.round(3))

[-6. -6.  0.  0.  0.  0.  6.  6.]

[[ 3.+0.j  0.+0.j  0.+0.j  0.-3.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j -3.+0.j  0.+3.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.-3.j -3.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+3.j  0.+0.j  0.+0.j  3.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j  3.+0.j  0.+0.j  0.+0.j  0.-3.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j -3.+0.j  0.+3.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.-3.j -3.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+3.j  0.+0.j  0.+0.j  3.+0.j]]


**Exercise 2:** Verify that the matrix produced is identical to the matrix obtained using numpy's kron function, and check that the eigenvalues match those produced by eigenspectrum.

In [12]:
import numpy

In [13]:
px = numpy.array([[0, 1],
                  [1, 0]])
py= numpy.array([[0, -1j],
                 [1j, 0]])
pz = numpy.array([[1,  0],
                  [0, -1]])
p0 = numpy.identity(2)

In [14]:
p0

array([[1., 0.],
       [0., 1.]])

In [15]:
qop1_mat = 3 * numpy.kron(p0, numpy.kron(px, py))
qop2_mat = numpy.kron(p0, numpy.kron(pz, pz))

In [16]:
matrix2 = qop1_mat + 3 * qop2_mat

In [17]:
print('OpenFermion and numpy matrices are the same: ', numpy.allclose(matrix, matrix2))

OpenFermion and numpy matrices are the same:  True


In [18]:
print('Eigenvalues of numpy matrix: ', numpy.linalg.eigvalsh(matrix2))

Eigenvalues of numpy matrix:  [-6. -6.  0.  0.  0.  0.  6.  6.]


## The FermionOperator class

OpenFermion can also represent operators on fermionic systems via the FermionicOperator class. This stores a representation of an operator as a sum of creation and annihilation operators. Individual FermionOperators are instantiated in a similar way to QubitOperators, but instead of needing to write a Pauli operator, we simply write the indices, and represent creation operators with a '^' symbol.

In [19]:
from openfermion import FermionOperator

fop1 = FermionOperator('2 1', 2)
print(fop1)
fop2 = FermionOperator('2^ 3', 3)
print()
print(fop1 * fop2)
print()
print(fop1 + 3 * fop2)

2 [2 1]

6 [2 1 2^ 3]

2 [2 1] +
9.0 [2^ 3]


FermionOperators introduce a new issue - individual creation and annihilation operators no longer commute, so there is not an immediately obvious choice for the 'standard form' that we would always want to write a product of these operators in --- nor is it immediately obvious whether two FermionOperators are equal!

One representation that is standard is 'normal ordering', where we first write all creation operators in decreasing order of their index and then all annihilation operators in decreasing order of their index. To convert a FermionOperator into its normal-ordered form, OpenFermion provides the normal_ordered function.

In [20]:
from openfermion import normal_ordered
print(fop1 * fop2)
print()
print(normal_ordered(fop1 * fop2))

6 [2 1 2^ 3]

6.0 [2^ 3 2 1] +
6.0 [3 1]


Note that what was a single term in its original order becomes a sum of two terms when converting to normal order! This is why normal order is not always preferable - the number of terms can grow quite large. Normal-ordering is also quite computationally costly to perform, and so openfermion does not perform it at each step. This implies that equality testing between FermionicOperators requires you to perform normal ordering first!

In [21]:
fop1 = FermionOperator('1 2', 1.0)
fop2 = FermionOperator('2 1', -1.0)
print(fop1 == fop2)
print(normal_ordered(fop1) == normal_ordered(fop2))
print(fop1 - fop2)
print(normal_ordered(fop1 - fop2))

False
True
1.0 [1 2] +
1.0 [2 1]
0


**Exercise 3:** Verify that FermionOperators obey the correct anti-commutation rules on a system with 2 indices.

In [22]:
from openfermion import anticommutator

In [23]:
fop3 = FermionOperator('0^ 1', 1)
fop4 = FermionOperator('1 2^', 1)

In [24]:
anticommutator(fop3, fop4)

1 [0^ 1 1 2^] +
1.0 [1 2^ 0^ 1]

In [25]:
fop3 * fop4 + fop4 * fop3

1 [0^ 1 1 2^] +
1.0 [1 2^ 0^ 1]

In [26]:
normal_ordered(fop3 * fop4 + fop4 * fop3)

0

In [27]:
fop5 = FermionOperator('0^ 1')
fop6 = FermionOperator('1^ 0')

In [28]:
(fop5 * fop6 + fop6 * fop5)

1.0 [0^ 1 1^ 0] +
1.0 [1^ 0 0^ 1]

In [29]:
normal_ordered(fop5 * fop6 + fop6 * fop5)

1.0 [0^ 0] +
2.0 [1^ 0^ 1 0] +
1.0 [1^ 1]

The above operations to generate a sparse matrix and eigenspectrum work just as well for fermion operators as they do qubit operators. However, remember that creation and annihilation operators are not themselves hermitian. To create an observable operator with real eigenspectrum, we should sum our fermion operators with their hermitian conjugates. OpenFermion provides the functions hermitian_conjugated and is_hermitian to help with this.

In [30]:
from openfermion import hermitian_conjugated, is_hermitian

print(is_hermitian(fop1))
print()
hermitian_op = fop1 + hermitian_conjugated(fop1)
print(hermitian_op)
print()
print(is_hermitian(hermitian_op))
print()
matrix = get_sparse_operator(hermitian_op)
print(matrix.todense().round(2))
print()
print(eigenspectrum(fop1 + hermitian_conjugated(fop1)))

False

1.0 [1 2] +
1.0 [2^ 1^]

True

[[ 0.+0.j  0.+0.j  0.+0.j -1.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [-1.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j -1.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j -1.+0.j  0.+0.j  0.+0.j  0.+0.j]]

[-1. -1.  0.  0.  0.  0.  1.  1.]


**Exercise 4:** Investigate the matrix representation of the sum of a single creation and single annihilation operator. What does this remind you of?

In [31]:
ferop0 = FermionOperator('0') + FermionOperator('0^')
ferop1 = FermionOperator('1') + FermionOperator('1^')

In [32]:
matrix_ferop0 = get_sparse_operator(ferop0)
matrix_ferop1 = get_sparse_operator(ferop1)

In [33]:
print(matrix_ferop0.todense().round(2))

[[0.+0.j 1.+0.j]
 [1.+0.j 0.+0.j]]


In [34]:
print(matrix_ferop1.todense().round(2))

[[ 0.+0.j  1.+0.j  0.+0.j  0.+0.j]
 [ 1.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j -1.+0.j]
 [ 0.+0.j  0.+0.j -1.+0.j  0.+0.j]]


Those matrices look a lot like Pauli operators

In [35]:
print('Is operator {} \n the same matrix as X: {}'.format(ferop0, 
                                                          numpy.allclose(matrix_ferop0.todense(), px)))

Is operator 1.0 [0] +
1.0 [0^] 
 the same matrix as X: True


In [36]:
print('Is operator {} \n the same matrix as Z0X1: {}'.format(ferop1, 
                                                          numpy.allclose(matrix_ferop1.todense(),
                                                                         numpy.kron(pz, px))))

Is operator 1.0 [1] +
1.0 [1^] 
 the same matrix as Z0X1: True


## The Jordan-Wigner transformation

Transforming between fermionic and qubit operators is a somewhat complicated task, as one needs to preserve the commutation relations of individual operators. Many choices for this transformation exist, all with various advantages and disadvantages. We will study this shortly in both the lectures and tutorials, but for the sake of this tutorial we will just use one possible transformation - the Jordan-Wigner transformation.

In short, the Jordan-Wigner transformation maps $$\hat{c}_i\rightarrow\frac{1}{2}\otimes_{j<i}Z_j\otimes(X_i+iY_i)$$
$$\hat{c}^{\dagger}_i\rightarrow\frac{1}{2}\otimes_{j<i}Z_j\otimes(X_i-iY_i)$$
This is implemented in OpenFermion by the jordan_wigner function.

In [37]:
from openfermion import jordan_wigner
print('Mapping annihilation operator')
print(jordan_wigner(FermionOperator('5')))
print()
print('Mapping creation operator')
print(jordan_wigner(FermionOperator('5^')))

Mapping annihilation operator
0.5 [Z0 Z1 Z2 Z3 Z4 X5] +
0.5j [Z0 Z1 Z2 Z3 Z4 Y5]

Mapping creation operator
0.5 [Z0 Z1 Z2 Z3 Z4 X5] +
-0.5j [Z0 Z1 Z2 Z3 Z4 Y5]


**Exercise 5:** Check that the Jordan-Wigner transform preserves the fermionic commutation relations on a $2$ site system.

In [38]:
fop0 = FermionOperator('0', 1.0)
fop0dag = FermionOperator('0^', 1.0)
fop1 = FermionOperator('1', 1.0)
fop1dag = FermionOperator('1^', 1.0)

In [39]:
qop0 = jordan_wigner(fop0)
qop0dag = jordan_wigner(fop0dag)
qop1 = jordan_wigner(fop1)
qop1dag = jordan_wigner(fop1dag)

In [40]:
# The anticommutator of creation annihilation on same site
print('Fermionic anticommutator in site 0: ', normal_ordered(anticommutator(fop0, fop0dag)))
print('Jordan-Wigner transformed anticommutator: ', anticommutator(qop0, qop0dag))
print()
print('Fermionic anticommutator in site 1: ', normal_ordered(anticommutator(fop1, fop1dag)))
print('Jordan-Wigner transformed anticommutator: ', anticommutator(qop1, qop1dag))

Fermionic anticommutator in site 0:  1.0 []
Jordan-Wigner transformed anticommutator:  (1+0j) []

Fermionic anticommutator in site 1:  1.0 []
Jordan-Wigner transformed anticommutator:  (1+0j) []


In [41]:
# The anticommutator of creation annihilation on different sites
print('Fermionic anticommutator 0 1^: ', normal_ordered(anticommutator(fop0, fop1dag)))
print('Jordan-Wigner transformed anticommutator: ', anticommutator(qop0, qop1dag))
print()
print('Fermionic anticommutator 0^ 1^: ', normal_ordered(anticommutator(fop0dag, fop1dag)))
print('Jordan-Wigner transformed anticommutator: ', anticommutator(qop0dag, qop1dag))

Fermionic anticommutator 0 1^:  0
Jordan-Wigner transformed anticommutator:  0

Fermionic anticommutator 0^ 1^:  0
Jordan-Wigner transformed anticommutator:  0


We leave as an exercise to check the rest of anticommutation relations between 0 and 1

**Exercise 6:** Recall the conditions for a QubitOperator to be Hermitian, and check that this is preserved under the Jordan-Wigner transformation.

In [42]:
from openfermion import hermitian_conjugated, is_hermitian

In [43]:
# A QubitOperator is Hermitian if its coefficients are real
print('Is {} hermitian: {}'.format(qop0, is_hermitian(qop0)))

Is 0.5 [X0] +
0.5j [Y0] hermitian: False


In [44]:
print('Is {} hermitian: {}'.format(qop0+hermitian_conjugated(qop0),
                                   is_hermitian(qop0+hermitian_conjugated(qop0))))

Is 1.0 [X0] hermitian: True


**Exercise 7:** Check that the Jordan-Wigner transform preserves the spectrum of the operator $\hat{c}^{\dagger}_0\hat{c}_1+\hat{c}^{\dagger}_1\hat{c}_0$.

In [45]:
toy_hamiltonian = fop0dag*fop1 + fop1dag*fop0
toy_ham_jw = jordan_wigner(toy_hamiltonian)
print(toy_hamiltonian)
print()
print(toy_ham_jw)

1.0 [0^ 1] +
1.0 [1^ 0]

(0.5+0j) [X0 X1] +
(0.5+0j) [Y0 Y1]


In [46]:
print('Spectrum of Fermionic toy hamiltonian: {}'.format(eigenspectrum(toy_hamiltonian)))

Spectrum of Fermionic toy hamiltonian: [-1.  0.  0.  1.]


In [47]:
print('Spectrum of Fermionic toy hamiltonian: {}'.format(eigenspectrum(toy_ham_jw)))

Spectrum of Fermionic toy hamiltonian: [-1.  0.  0.  1.]


# Quantum Chemistry

One of the most critical functions of OpenFermion is its ability to interface with computational chemistry packages such as psi4, pySCF, and DIRAC, which are then used to provide data (e.g. Hamiltonians) on molecular systems. This is done through various interface packages; due to the difficulty in installing these packages we will not use them here. However, OpenFermion ships with some molecular system data already, which is enough for us to use.

## The MolecularData class

OpenFermion uses the MolecularData class to store data from computational chemistry packages. This class has additionally a load and save function, as this data often takes a significant amount of time to prepare. To load stored data, we need to state the geometry of the molecule (in cartesian co-ordinates), the basis in which the molecule is stored, and its charge and multiplicity (i.e. $2n+1$, where $n$ is the number of unpaired spins). OpenFermion also requests a description of the molecule studied

In [48]:
from openfermion import MolecularData

diatomic_bond_length = .7414

geometry = [('H', (0., 0., 0.)), 
            ('H', (0., 0., diatomic_bond_length))]

basis = 'sto-3g'
multiplicity = 1
charge = 0
description = format(diatomic_bond_length)

molecule = MolecularData(
    geometry,
    basis,
    multiplicity,
    description=description)
molecule.load()

print(molecule.name)

H2_sto-3g_singlet_0.7414


In addition to the above data, and being able to generate the Hamiltonian of the target molecule, the MolecularData class contains various information about classical computational approximations of the ground state energy.

In [49]:
print("Bond Length in Angstroms: {}".format(diatomic_bond_length))
print("Hartree Fock (mean-field) energy in Hartrees: {}".format(molecule.hf_energy))
print("FCI (Exact) energy in Hartrees: {}".format(molecule.fci_energy))

Bond Length in Angstroms: 0.7414
Hartree Fock (mean-field) energy in Hartrees: -1.116684386906734
FCI (Exact) energy in Hartrees: -1.137270174625328


## The InteractionOperator class

The Hamiltonian of the above system may be obtained via the get_molecular_hamiltonian function

In [50]:
hamiltonian = molecule.get_molecular_hamiltonian()
print(hamiltonian.__class__)
print(hamiltonian)

<class 'openfermion.ops._interaction_operator.InteractionOperator'>
() 0.713753990544915
((0, 1), (0, 0)) -1.2524635715927757
((1, 1), (1, 0)) -1.2524635715927757
((2, 1), (2, 0)) -0.4759487172683097
((3, 1), (3, 0)) -0.4759487172683097
((0, 1), (0, 1), (0, 0), (0, 0)) 0.3372443828669511
((0, 1), (0, 1), (2, 0), (2, 0)) 0.09064440419713085
((0, 1), (1, 1), (1, 0), (0, 0)) 0.3372443828669511
((0, 1), (1, 1), (3, 0), (2, 0)) 0.09064440419713085
((0, 1), (2, 1), (0, 0), (2, 0)) 0.09064440419713081
((0, 1), (2, 1), (2, 0), (0, 0)) 0.33173404792821903
((0, 1), (3, 1), (1, 0), (2, 0)) 0.09064440419713081
((0, 1), (3, 1), (3, 0), (0, 0)) 0.33173404792821903
((1, 1), (0, 1), (0, 0), (1, 0)) 0.3372443828669511
((1, 1), (0, 1), (2, 0), (3, 0)) 0.09064440419713085
((1, 1), (1, 1), (1, 0), (1, 0)) 0.3372443828669511
((1, 1), (1, 1), (3, 0), (3, 0)) 0.09064440419713085
((1, 1), (2, 1), (0, 0), (3, 0)) 0.09064440419713081
((1, 1), (2, 1), (2, 0), (1, 0)) 0.33173404792821903
((1, 1), (3, 1), (1, 0), 

The above operator is returned neither as a FermionOperator nor a QubitOperator, but an InteractionOperator. This is similar to a FermionOperator, but has the restriction that it contains only one-body and two-body terms that conserve particle number and spin. This allows for various speedups in calculation not available to the more general class.

Convering an InteractionOperator to a FermionOperator is performed using the get_fermion_operator function

In [51]:
from openfermion import get_fermion_operator
fermionic_hamiltonian = get_fermion_operator(hamiltonian)
print(fermionic_hamiltonian)
print()
qubit_hamiltonian = jordan_wigner(fermionic_hamiltonian)
print(qubit_hamiltonian)

0.713753990544915 [] +
-1.2524635715927757 [0^ 0] +
0.3372443828669511 [0^ 0^ 0 0] +
0.09064440419713085 [0^ 0^ 2 2] +
0.3372443828669511 [0^ 1^ 1 0] +
0.09064440419713085 [0^ 1^ 3 2] +
0.09064440419713081 [0^ 2^ 0 2] +
0.33173404792821903 [0^ 2^ 2 0] +
0.09064440419713081 [0^ 3^ 1 2] +
0.33173404792821903 [0^ 3^ 3 0] +
0.3372443828669511 [1^ 0^ 0 1] +
0.09064440419713085 [1^ 0^ 2 3] +
-1.2524635715927757 [1^ 1] +
0.3372443828669511 [1^ 1^ 1 1] +
0.09064440419713085 [1^ 1^ 3 3] +
0.09064440419713081 [1^ 2^ 0 3] +
0.33173404792821903 [1^ 2^ 2 1] +
0.09064440419713081 [1^ 3^ 1 3] +
0.33173404792821903 [1^ 3^ 3 1] +
0.33173404792821914 [2^ 0^ 0 2] +
0.0906444041971308 [2^ 0^ 2 0] +
0.33173404792821914 [2^ 1^ 1 2] +
0.0906444041971308 [2^ 1^ 3 0] +
-0.4759487172683097 [2^ 2] +
0.09064440419713085 [2^ 2^ 0 0] +
0.34869688341114263 [2^ 2^ 2 2] +
0.09064440419713085 [2^ 3^ 1 0] +
0.34869688341114263 [2^ 3^ 3 2] +
0.33173404792821914 [3^ 0^ 0 3] +
0.0906444041971308 [3^ 0^ 2 1] +
0.33173404792

**Exercise 8:** Check that the above operations preserve the eigenspectrum of the H2 Hamiltonian

In [52]:
print('Eigenspectrum of H2 in FermionOperator: \n {}'.format(eigenspectrum(fermionic_hamiltonian)))

Eigenspectrum of H2 in FermionOperator: 
 [-1.13727017 -0.53870958 -0.53870958 -0.53247901 -0.53247901 -0.53247901
 -0.44698572 -0.44698572 -0.16990139  0.23780527  0.23780527  0.35243413
  0.35243413  0.47983611  0.71375399  0.92010671]


In [53]:
print('Eigenspectrum of H2 in QubitOperator: \n {}'.format(eigenspectrum(qubit_hamiltonian)))

Eigenspectrum of H2 in QubitOperator: 
 [-1.13727017 -0.53870958 -0.53870958 -0.53247901 -0.53247901 -0.53247901
 -0.44698572 -0.44698572 -0.16990139  0.23780527  0.23780527  0.35243413
  0.35243413  0.47983611  0.71375399  0.92010671]


In [54]:
print('Are the spectrum equal: {}'.format(numpy.allclose(eigenspectrum(fermionic_hamiltonian),
                                                         eigenspectrum(qubit_hamiltonian))))

Are the spectrum equal: True


# The OpenFermion-Cirq interface

In the VQE context, the Hamiltonians $H$ we created above above generate cost functions $f(\vec{\theta})=\langle\Psi(\vec{\theta})|H|\Psi(\vec{\theta})\rangle$ for classical optimization. The openfermioncirq package provides an interface between cirq (for generating and simulating circuits) and openfermion (for generating Hamiltonians) to provide precisely this functionality.

As a simple example (before coming back to H$_2$), let us study a trivial VQE on a single qubit - optimize the ansatz $|\Psi(\theta)\rangle=e^{i\theta Y}|0\rangle$ for the Hamiltonian $H=Z+X$.

In openfermioncirq, a VQE experiment is contained within a openfermioncirq.VariationalStudy class. This requires as input a name, an ansatz, and a cost function.

The cost function can be generated from our hamiltonian using the openfermioncirq.HamiltonianObjective class.

The variational ansatz itself must extend the openfermioncirq.VariationalAnsatz class (i.e. be a subclass). In doing so, it needs to contain the following methods:

- params: a function that returns the parameters to be optimized (in a list/tuple)
- operations: a generator for the variational circuit that takes as input a list of qubits (note - a generator, not the final cirq.Circuit - this is typically cleaner)
- _generate_qubits: a function that produces a list/tuple containing the qubits in order.
(Note, the order of the qubits will correspond to the index order in the openfermion Hamiltonian)

**Exercise 9:** use cirq, openfermion, and openfermioncirq to create the above variational ansatz (remember to include a free parameter using sympy!)

In [55]:
# To describe: VariationalStudy, HamiltonianObjective, OptimizationParams
# Toy example - VQE on a single qubit!
import cirq
import sympy
import openfermioncirq

theta = sympy.Symbol('theta')

class HelloWorldsAnsatz(openfermioncirq.VariationalAnsatz):
    
    def params(self):
        return [theta]

    def operations(self, qubits):
        yield cirq.ry(theta).on(qubits[0])
        
    def _generate_qubits(self):
        return [cirq.GridQubit(0,0)]

ansatz = HelloWorldsAnsatz()

hamiltonian = QubitOperator('X0') + QubitOperator('Z0')

objective = openfermioncirq.HamiltonianObjective(hamiltonian)
study = openfermioncirq.VariationalStudy(
    name='Hello, worlds!',
    ansatz=ansatz,
    objective=objective)

print(study.circuit)

(0, 0): ───Ry(theta)───


To actually simulate (or run) the VQE, we need to define the optimization strategy. Typically for digital quantum simulation, this is performed via gradient descent or gradient-free optimization methods, of which there are many popular choices. All typically require an initial guess --- quite often in a VQE the guess should be slightly off-set to prevent starting in a local maximum or saddle point.

Openfermion summarizes these in a neat form in the openfermioncirq.optimization package, which has the OptimizationParams class (basically a container for optimization metaparameters) and various wrappers for calls to scipy routines. For example, let's optimize the above VQE using the COBYLA algorithm

In [56]:
from openfermioncirq.optimization import COBYLA, OptimizationParams
optimization_params = OptimizationParams(
    algorithm=COBYLA,
    initial_guess=[0.01])
result = study.optimize(optimization_params)
print('Optimized VQE result: {}'.format(result.optimal_value))
print('Target Hamiltonian eigenvalues: {}'.format(eigenspectrum(hamiltonian)))

Optimized VQE result: -1.4142135527847444
Target Hamiltonian eigenvalues: [-1.41421356  1.41421356]


# The unitary coupled cluster ansatz

The unitary coupled cluster ansatz is probably the most well-known variational ansatz. We will study this ansatz in more detail later in class, but after going through the above you can hopefully implement it for H$_2$.

In short, the unitary coupled cluster ansatz takes the form

$$ e^{T(\vec{\theta})-T^{\dagger}(\vec{\theta})}|111\ldots000\ldots\rangle $$,

where in the starting state the first $\eta$ spin-orbitals contain an electron and the remaining $N-\eta$ sites are empty, and the cluster operator $T$ contains terms which excite from empty orbitals to filled. This operator is commonly truncated to single operators $\theta^i_j\hat{c}_i^{\dagger}\hat{c}_j$ (where $i$ is empty and $j$ is filled) and double operators $\theta^{i,j}_{k,l}\hat{c}_i^{\dagger}\hat{c}_j^{\dagger}\hat{c}_k\hat{c}_l$. (where $i$ and $j$ are empty and $k$ and $l$ are filled).

In the H$_2$ example above (because of the small basis size), we only have two empty and two filled orbitals. Moreover, due to symmetry constraints it turns out that the ground state contains no single excitations, and so the ansatz takes the form of a single term (we can drop the indices on $\theta$ for brevity)

$$ e^{\theta\hat{c}^{\dagger}_3\hat{c}^{\dagger}_2\hat{c}_1\hat{c}_0-\mathrm{h.c.}}|1100\rangle $$

**Exercise 9:** convert the cluster operator $T-T^{\dagger}$ for H$_2$ into qubit form using the Jordan-Wigner transform via openfermion. (Check your result with pen and paper.)

In [57]:
# We just create a cluster operator and its Hermitian conjugate with value 1
# later we will add a parameter to it.
t_fop = FermionOperator('0^ 1^ 2 3')
t_fopdag = hermitian_conjugated(t_fop)
print('Cluster operators: ', t_fop, t_fopdag)

Cluster operators:  1.0 [0^ 1^ 2 3] 1.0 [3^ 2^ 1 0]


In [58]:
ucc_qop = jordan_wigner(t_fop - t_fopdag)
print('Unitary coupled-cluster JW: \n', ucc_qop)

Unitary coupled-cluster JW: 
 -0.125j [X0 X1 X2 Y3] +
-0.125j [X0 X1 Y2 X3] +
0.125j [X0 Y1 X2 X3] +
-0.125j [X0 Y1 Y2 Y3] +
0.125j [Y0 X1 X2 X3] +
-0.125j [Y0 X1 Y2 Y3] +
0.125j [Y0 Y1 X2 Y3] +
0.125j [Y0 Y1 Y2 X3]


**Exercise 10:** Write a function to generate the circuit for $\exp(\theta c_0^{\dagger}c_1^{\dagger}c_2c_3-\mathrm{h.c.})$ with a free angle $\theta$. Use this to create a openfermioncirq.VariationalAnsatz.


In [59]:
from openfermion import count_qubits

In [66]:
def qubitop_exponent_to_cirq(operator_list, *qubits, parameter):
    '''
    Exponentiate a list of QubitOperator to a cirq.
    
    Exponentiate a list of QubitOperators to yield a cirq circuit generator.
    
    Args:
        operator_list(QubitOperator, list): Operators to transform to a circuit.
        qubits(cirq.qubits): List of cirq qubits.
    
    Yields:
        cirq circuit generator
    '''
    if not isinstance(operator_list, (list, QubitOperator)):
        raise TypeError('Input must be a list or QubitOperator')
    if isinstance(operator_list, QubitOperator):
        operator_list = list(operator_list)
        
    rot_dic = {'X':lambda q,s: cirq.ry(-1*s*numpy.pi/2).on(q),
               'Y':lambda q,s: cirq.rx(-1*s*numpy.pi/2).on(q),
               'Z':lambda q,s: cirq.I.on(q)}
    
    qubit_list=[*qubits]
    
    for i, operator in enumerate(operator_list):
        paulis = list(operator.terms.keys())[0]
        sign = numpy.sign(list(operator.terms.values())[0])
        
        for qbt, pau in paulis:
            yield rot_dic[pau](qubit_list[qbt], 1)

        if len(paulis) > 1:
            for j in range(len(paulis) - 1):
                yield cirq.CNOT(qubit_list[paulis[j][0]],
                                qubit_list[paulis[j+1][0]])

        yield cirq.rz(sign*parameter[0]).on(qubit_list[paulis[-1][0]])
        
        if len(paulis) > 1:
            for j in range(len(paulis)-1, 0, -1):
                yield cirq.CNOT(qubit_list[paulis[j-1][0]],
                                qubit_list[paulis[j][0]])
        
        for qbt, pau in paulis:
            yield rot_dic[pau](qubit_list[qbt], -1)

In [67]:
class UCCAnstaz(openfermioncirq.VariationalAnsatz):
    def __init__(self, operators, num_qubits, num_params):
        '''
        Start variational ansatz class.
        
        Args:
            operators(QubitOperator): UCCSD Pauli operators.
            num_qubits(int): Number of qubits to create the circuit.
            num_params(int): Number of parameters.
            
        Note:
            If one wants to be accurate when using CCSD amplitudes
            as parameters for UCCSD, the number of free parameters
            is much smaller than the number of Pauli operators.
            Hence one needs to specify how many parameters are allowed.
            Beawere that parameters are added by module num_params.
        '''
        self.operators = operators
        self.num_qubits = num_qubits
        self.num_params = num_params
        super().__init__(qubits = cirq.LineQubit.range(num_qubits))
    
    def params(self):
        return [sympy.Symbol('theta_'+str(i))
                for i in range(self.num_params)]
        
    def _generate_qubits(self):
        pass
    
    def operations(self, qubits):
        yield qubitop_exponent_to_cirq(self.operators, *qubits,
                                       parameter=self.params())

In [68]:
ansatz=UCCAnstaz(list(ucc_qop), count_qubits(ucc_qop), 1)

In [69]:
ansatz.circuit

0: ───Rx(-0.5π)───@─────────────────────────────────────────────@───────────Rx(0.5π)────Rx(-0.5π)───@────────────────────────────────────────────@───────────Rx(0.5π)────Rx(-0.5π)───@────────────────────────────────────────────@───────────Rx(0.5π)────Rx(-0.5π)───@────────────────────────────────────────────@───────────Rx(0.5π)────Ry(-0.5π)───@─────────────────────────────────────────────@───────────Ry(0.5π)────Ry(-0.5π)───@─────────────────────────────────────────────@───────────Ry(0.5π)────Ry(-0.5π)───@─────────────────────────────────────────────@───────────Ry(0.5π)────Ry(-0.5π)───@────────────────────────────────────────────@──────────Ry(0.5π)───
                  │                                             │                                   │                                            │                                   │                                            │                                   │                                            │                                   │                                             │                                   │                                             │                                   │                                             │                                   │                                            │
1: ───Ry(-0.5π)───X───@──────────────────────────────@──────────X───────────Ry(0.5π)────Ry(-0.5π)───X───@─────────────────────────────@──────────X───────────Ry(0.5π)────Rx(-0.5π)───X───@─────────────────────────────@──────────X───────────Rx(0.5π)────Rx(-0.5π)───X───@─────────────────────────────@──────────X───────────Rx(0.5π)────Ry(-0.5π)───X───@──────────────────────────────@──────────X───────────Ry(0.5π)────Ry(-0.5π)───X───@──────────────────────────────@──────────X───────────Ry(0.5π)────Rx(-0.5π)───X───@──────────────────────────────@──────────X───────────Rx(0.5π)────Rx(-0.5π)───X───@─────────────────────────────@──────────X──────────Rx(0.5π)───
                      │                              │                                                  │                             │                                                  │                             │                                                  │                             │                                                  │                              │                                                  │                              │                                                  │                              │                                                  │                             │
2: ───Rx(-0.5π)───────X───@──────────────────────@───X──────────Rx(0.5π)────Ry(-0.5π)───────────────────X───@─────────────────────@───X──────────Ry(0.5π)────Rx(-0.5π)───────────────────X───@─────────────────────@───X──────────Rx(0.5π)────Ry(-0.5π)───────────────────X───@─────────────────────@───X──────────Ry(0.5π)────Rx(-0.5π)───────────────────X───@──────────────────────@───X──────────Rx(0.5π)────Ry(-0.5π)───────────────────X───@──────────────────────@───X──────────Ry(0.5π)────Rx(-0.5π)───────────────────X───@──────────────────────@───X──────────Rx(0.5π)────Ry(-0.5π)───────────────────X───@─────────────────────@───X──────────Ry(0.5π)──────────────
                          │                      │                                                          │                     │                                                          │                     │                                                          │                     │                                                          │                      │                                                          │                      │                                                          │                      │                                                          │                     │
3: ───Rx(-0.5π)───────────X───Rz(-1.0*theta_0)───X───Rx(0.5π)───Ry(-0.5π)───────────────────────────────────X───Rz(1.0*theta_0)───X───Ry(0.5π)───Ry

**Exercise 11:** Write a function to prepare the computational basis state $c_1^{\dagger}c_0^{\dagger}|0000\rangle$.

In [70]:
def initial_state_prepartion(qubits, positions):
    return cirq.Circuit(cirq.X(qubits[i]) for i in positions)

In [71]:
initial_state_prepartion(ansatz.qubits, positions=[0,1])

0: ───X───

1: ───X───

**Exercise 12:** Create a study of the H$_2$ molecule using OpenFermion-Cirq, optimize the resulting VQE, and compare your results to the Full-CI energy reported above.

In [84]:
objective = openfermioncirq.HamiltonianObjective(qubit_hamiltonian)
study = openfermioncirq.VariationalStudy(
    name='UCC for H2',
    ansatz=ansatz,
    preparation_circuit=initial_state_prepartion(ansatz.qubits, [0,1]),
    objective=objective)

In [85]:
study.circuit

0: ───X───Rx(-0.5π)───@─────────────────────────────────────────────@───────────Rx(0.5π)────Rx(-0.5π)───@────────────────────────────────────────────@───────────Rx(0.5π)────Rx(-0.5π)───@────────────────────────────────────────────@───────────Rx(0.5π)────Rx(-0.5π)───@────────────────────────────────────────────@───────────Rx(0.5π)────Ry(-0.5π)───@─────────────────────────────────────────────@───────────Ry(0.5π)────Ry(-0.5π)───@─────────────────────────────────────────────@───────────Ry(0.5π)────Ry(-0.5π)───@─────────────────────────────────────────────@───────────Ry(0.5π)────Ry(-0.5π)───@────────────────────────────────────────────@──────────Ry(0.5π)───
                      │                                             │                                   │                                            │                                   │                                            │                                   │                                            │                                   │                                             │                                   │                                             │                                   │                                             │                                   │                                            │
1: ───X───Ry(-0.5π)───X───@──────────────────────────────@──────────X───────────Ry(0.5π)────Ry(-0.5π)───X───@─────────────────────────────@──────────X───────────Ry(0.5π)────Rx(-0.5π)───X───@─────────────────────────────@──────────X───────────Rx(0.5π)────Rx(-0.5π)───X───@─────────────────────────────@──────────X───────────Rx(0.5π)────Ry(-0.5π)───X───@──────────────────────────────@──────────X───────────Ry(0.5π)────Ry(-0.5π)───X───@──────────────────────────────@──────────X───────────Ry(0.5π)────Rx(-0.5π)───X───@──────────────────────────────@──────────X───────────Rx(0.5π)────Rx(-0.5π)───X───@─────────────────────────────@──────────X──────────Rx(0.5π)───
                          │                              │                                                  │                             │                                                  │                             │                                                  │                             │                                                  │                              │                                                  │                              │                                                  │                              │                                                  │                             │
2: ───────Rx(-0.5π)───────X───@──────────────────────@───X──────────Rx(0.5π)────Ry(-0.5π)───────────────────X───@─────────────────────@───X──────────Ry(0.5π)────Rx(-0.5π)───────────────────X───@─────────────────────@───X──────────Rx(0.5π)────Ry(-0.5π)───────────────────X───@─────────────────────@───X──────────Ry(0.5π)────Rx(-0.5π)───────────────────X───@──────────────────────@───X──────────Rx(0.5π)────Ry(-0.5π)───────────────────X───@──────────────────────@───X──────────Ry(0.5π)────Rx(-0.5π)───────────────────X───@──────────────────────@───X──────────Rx(0.5π)────Ry(-0.5π)───────────────────X───@─────────────────────@───X──────────Ry(0.5π)──────────────
                              │                      │                                                          │                     │                                                          │                     │                                                          │                     │                                                          │                      │                                                          │                      │                                                          │                      │                                                          │                     │
3: ───────Rx(-0.5π)───────────X───Rz(-1.0*theta_0)───X───Rx(0.5π)───Ry(-0.5π)───────────────────────────────────X───Rz(1.0*

In [86]:
optimization_params = OptimizationParams(
    algorithm=COBYLA,
    initial_guess=numpy.random.rand(ansatz.num_params))
result = study.optimize(optimization_params)
print('Optimized VQE result: {}'.format(result.optimal_value))

Optimized VQE result: -1.1372700216580982


In [87]:
print('Full configuration-interaction energy: {}'.format(molecule.fci_energy))

Full configuration-interaction energy: -1.137270174625328


In [88]:
print('Optimized parameters: \n',result.optimal_parameters)

Optimized parameters: 
 [0.75705424]


### Repeat the same study for different bond distances

**Exercise 13:** Openfermion also contains data for the H$_2$ molecule at diatomic bond lengths from 0.3 to 2.5 Angstrom in increments of 0.1 Angstrom. Repeat the above study for all these curves, and plot the bond dissociation curve (a plot of the ground state energy as a function of the bond length) for the H$_2$ molecule. Compare to the FCI results.

**Exercise 14:** The UCC circuit that you generated is horribly long - how much can you optimize it?

In [71]:
# This is left as an exercise for the students